In [50]:
import tensorflow as tf
tf.random.set_seed(10)
import tensorflow_datasets as tfds
dataset, dataset_info = tfds.load(name='malaria', shuffle_files=True, with_info=True, as_supervised=True, split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'])

In [51]:
# Preprocessing function
def preprocess(image, label):
    image = tf.image.resize(image, [224, 224])  # Resize images to 224x224
    image = tf.cast(image, tf.float32)  # Convert images to float32
    return image, label

# Apply preprocessing
train_dataset, valid_dataset, test_dataset = dataset

train_dataset = train_dataset.map(preprocess)
valid_dataset = valid_dataset.map(preprocess)
test_dataset = test_dataset.map(preprocess)

# Normalization function
def normalise(image, label):
    return image / 255, label

# Apply normalization
train_dataset = train_dataset.map(normalise)
valid_dataset = valid_dataset.map(normalise)
test_dataset = test_dataset.map(normalise)

# Apply shuffling, batching, and prefetching
train_dataset = train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
valid_dataset = valid_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

In [52]:
from tensorflow.keras.layers import Dense, Conv2D, InputLayer, RandomRotation, RandomFlip, Resizing, CenterCrop, BatchNormalization, Flatten, Dropout
from tensorflow.keras.models import Sequential

model = Sequential()

# Input Layer
model.add(InputLayer(shape=(224, 224, 3)))

# Data Augmentation Layers
model.add(Resizing(224, 224))
model.add(RandomRotation(0.2))  # Randomly rotate images
model.add(RandomFlip('horizontal'))  # Flip images horizontally
model.add(CenterCrop(200, 200))

# Conv Layers
model.add(Conv2D(filters=5, kernel_size=3, padding='same', strides=2, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=10, kernel_size=3, padding='same', strides=2, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=20, kernel_size=3, padding='valid', strides=2, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=25, kernel_size=3, padding='valid', strides=2, activation='relu'))
model.add(BatchNormalization())

# Global Pooling Layer
model.add(Flatten())

# Fully Connected Layers
model.add(Dense(units=100, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))  # Add dropout to reduce overfitting
model.add(Dense(units=400, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(units=100, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(units=1, activation='sigmoid'))

# Model Summary
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resizing_2 (Resizing)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation_2               │ (None, 224, 224, 3)    │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_flip_2 (RandomFlip)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ center_crop_2 (CenterCrop)      │ (None, 200, 200, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 100, 100, 5)    │           140 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 100, 100, 5)    │            20 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 50, 50, 10)     │           460 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 50, 50, 10)     │            40 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 24, 24, 20)     │         1,820 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 24, 24, 20)     │            80 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 11, 11, 25)     │         4,525 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_17          │ (None, 11, 11, 25)     │           100 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 3025)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 100)            │       302,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 100)            │           400 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 400)            │        40,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_19          │ (None, 400)            │         1,600 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 400)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 100)            │        40,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_20          │ (None, 100)            │           400 │
│ (BatchNormalization)            │                        │             

 Total params: 392,786 (1.50 MB)

 Trainable params: 391,466 (1.49 MB)

 Non-trainable params: 1,320 (5.16 KB)

# Custom training

In [54]:
from tensorflow.keras.losses import Loss
from tensorflow.keras.metrics import BinaryAccuracy,Metric

class CustomBCE(Loss):
    def __init__(self,factor=1):
        super(CustomBCE,self).__init__()
        self.factor = factor
    def call(self,y_true,y_pred):
        bce = BinaryCrossentropy()
        return bce(y_true,y_pred)*self.factor

class CustomBA(Metric):
    def __init__(self, factor=1, name="custom_binary_accuracy", **kwargs):
        super(CustomBA, self).__init__(name=name, **kwargs)
        self.factor = factor
        self.accuracy_fn = BinaryAccuracy()

    def update_state(self, y_true, y_pred,sample_weight=None):
        accuracy = self.accuracy_fn(y_true, y_pred) * self.factor
        return accuracy

    def result(self):
        return self.accuracy_fn.result()

    def reset_states(self):
        self.accuracy_fn.reset_states()

In [55]:
# %load_ext tensorboard
# %tensorboard --logdir=./logs --bind_all

In [56]:
OPTIMIZER = Adam()
METRIC = BinaryAccuracy()
METRIC_VAL = BinaryAccuracy()
@tf.function
def training_block(x_batch,y_batch):
    with tf.GradientTape() as recorder:
        y_pred = model(x_batch,training = True)
        loss = CustomBCE().call(y_batch,y_pred)
    partial_derivatives = recorder.gradient(loss,model.trainable_weights)
    OPTIMIZER.apply_gradients(zip(
        partial_derivatives,model.trainable_weights
    ))
    METRIC.update_state(y_batch, y_pred)
    return loss, METRIC.result()
@tf.function
def validation_block(x_batch,y_batch):
    y_pred = model(x_batch,training=False)
    loss = CustomBCE().call(y_batch,y_pred)
    METRIC_VAL.update_state(y_batch,y_pred)
    return loss,METRIC_VAL.result()

In [58]:
def customFit(train_dataset,valid_dataset,EPOCHS,model):
    for epoch in range(EPOCHS):
        tf.print(f"Training epoch {epoch+1}")
        METRIC.reset_state()
        for step, (x_batch,y_batch) in enumerate(train_dataset):
            loss, metric = training_block(x_batch,y_batch)
            if(step%100 == 0):
                print(f"Loss: {loss.numpy()}, Accuracy: {metric.numpy()}")
                
        METRIC_VAL.reset_state()
        for step, (x_batch,y_batch) in enumerate(valid_dataset):
            loss, metric = validation_block(x_batch,y_batch)
            if(step%30 == 0):
                print(f"Val Loss: {loss.numpy()}, Val Accuracy: {metric.numpy()}")

In [59]:
customFit(train_dataset,valid_dataset,5,model)

Training epoch 1
Loss: 0.09643975645303726, Accuracy: 0.96875
Loss: 0.09211443364620209, Accuracy: 0.9446163177490234


KeyboardInterrupt: 